Based on the code and the sample response data, I'll help you update the code to properly handle the received data. Here are the key changes needed:

```javascript:deploy1.3.3_combine/frontend/script.js
// ... existing code ...

document.getElementById('loadVideo').addEventListener('click', async () => {
    const videoLink = document.getElementById('videoLink').value.trim();
    const fileId = videoLink.match(/\/d\/(.+?)\//)[1];

    try {
        // Update to use the correct port (5000 instead of 3000)
        const response = await fetch(`http://localhost:5000/get_video_data?url=${videoLink}`);
        const data = await response.json();

        if (response.ok) {
            // Update video preview
            const previewLink = `https://drive.google.com/file/d/${fileId}/preview`;
            document.getElementById('video').src = previewLink;
            
            // Update transcript
            updateTranscript(data.transcript);
            
            // Parse criteria string to object
            const criteriaObj = JSON.parse(data.criteria);
            
            // Add event listeners - pass the criteria object directly
            addCriteriaListeners(criteriaObj, fileId);
        }
    } catch (error) {
        console.error('Error:', error);
        alert('Error loading video data');
    }
});

// Update the addCriteriaListeners function to handle the new criteria structure
function addCriteriaListeners(criteriaObj, videoId) {
    const buttons = document.querySelectorAll('.criteria-button');
    buttons.forEach(button => {
        button.addEventListener('click', () => {
            const criterionKey = button.getAttribute('data-criteria');
            currentCriteria = criterionKey;
            
            // Access the criterion directly from the parsed object
            const criterion = criteriaObj[criterionKey];
            
            if (criterion) {
                highlightTranscript(criterion);
                displayCriteriaInfo(criterion, criterionKey);
                displayCriteriaRecommendation(criterionKey, videoId);
            }
        });
    });
}

// ... existing code ...

async function displayCriteriaRecommendation(criterionKey, videoId) {
    const scoreElement = document.getElementById('criteriaScore');
    const reasonElement = document.getElementById('criteriaReason');

    try {
        const response = await fetch(`http://localhost:5000/get_video_data?url=https://drive.google.com/file/d/${videoId}/view`);
        const data = await response.json();

        if (response.ok) {
            const criteriaObj = JSON.parse(data.criteria);
            const criterion = criteriaObj[criterionKey];

            if (criterion && criterion.recommendationScore) {
                scoreElement.textContent = `Score: ${criterion.recommendationScore.score}`;
                reasonElement.innerHTML = `Reason: ${criterion.recommendationScore.reason}`;
            } else {
                scoreElement.textContent = 'Score: N/A';
                reasonElement.textContent = 'Reason: N/A';
            }
        }
    } catch (error) {
        console.error('Error:', error);
        scoreElement.textContent = 'Score: Error';
        reasonElement.textContent = 'Reason: Error loading data';
    }
}
```

Key changes made:
1. Updated the server port from 3000 to 5000 to match your curl example
2. Modified the criteria parsing to handle the direct JSON structure from the response
3. Updated the addCriteriaListeners function to work with the new criteria structure
4. Simplified the displayCriteriaRecommendation function to handle errors better and display the recommendation score and reason directly from the parsed data

The code now properly handles the response format shown in your curl example, where criteria is a JSON string that needs to be parsed and accessed directly by the criterion key.



Tôi sẽ giải thích những thay đổi chính trong code bằng tiếng Việt:

1. **Thay đổi port kết nối**:
```javascript
const response = await fetch(`http://localhost:5000/get_video_data?url=${videoLink}`);
```
- Chuyển từ port 3000 sang port 5000 để phù hợp với cấu hình server của bạn
- Đảm bảo API endpoint khớp với cấu hình backend

2. **Xử lý dữ liệu criteria**:
```javascript
// Parse criteria string to object
const criteriaObj = JSON.parse(data.criteria);
```
- Dữ liệu criteria từ server trả về dạng string nên cần parse thành object
- Chuyển đổi từ JSON string sang JavaScript object để dễ dàng truy cập

3. **Cập nhật hàm addCriteriaListeners**:
```javascript
function addCriteriaListeners(criteriaObj, videoId) {
    const buttons = document.querySelectorAll('.criteria-button');
    buttons.forEach(button => {
        button.addEventListener('click', () => {
            const criterionKey = button.getAttribute('data-criteria');
            currentCriteria = criterionKey;
            
            const criterion = criteriaObj[criterionKey];
            // ...
        });
    });
}
```
- Sửa lại cách truy cập dữ liệu criteria để phù hợp với cấu trúc mới
- Lấy thông tin criterion dựa trên key từ button
- Lưu lại criterion hiện tại để sử dụng sau này

4. **Cập nhật hàm displayCriteriaRecommendation**:
```javascript
async function displayCriteriaRecommendation(criterionKey, videoId) {
    try {
        // ... fetch data ...
        if (criterion && criterion.recommendationScore) {
            scoreElement.textContent = `Score: ${criterion.recommendationScore.score}`;
            reasonElement.innerHTML = `Reason: ${criterion.recommendationScore.reason}`;
        }
    } catch (error) {
        // Xử lý lỗi
    }
}
```
- Thêm xử lý lỗi khi không lấy được dữ liệu
- Hiển thị score và reason từ recommendationScore
- Sử dụng innerHTML cho reason vì có thể chứa HTML tags
- Thêm thông báo khi không có dữ liệu hoặc có lỗi

5. **Cải thiện xử lý lỗi**:
- Thêm try-catch để bắt lỗi khi gọi API
- Hiển thị thông báo lỗi rõ ràng cho người dùng
- Log lỗi ra console để dễ debug

Những thay đổi này giúp:
- Code chạy ổn định hơn với cấu trúc dữ liệu mới
- Xử lý lỗi tốt hơn
- Hiển thị dữ liệu chính xác hơn
- Dễ bảo trì và debug hơn